In [ ]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from QuantumRingsLib import qasm2
from QuantumRingsLib import Parameter, ParameterVector

from random import random, uniform
from matplotlib import pyplot as plt
import numpy as np
import math
import time

import scipy
from scipy import optimize
import os
import math

provider = QuantumRingsProvider(token=os.environ.get('TOKEN_QUANTUMRINGS'), name=os.environ.get('ACCOUNT_QUANTUMRINGS'))
backend = provider.get_backend("scarlet_quantum_rings")
shots = 2

print(provider.active_account())

In [2]:
# Implements a simple ansatz circuit using RY rotations
# Inputs: qc – QuantumCircuit
# theta_list – parameter list must be (reps + 1) * n_qubits wide.
# q – the qubit register to use
# n_qubits – Number of qubits to use
# reps – repeats of the circuit, defaults to 5
# insert_barriers –  whether to create a barrier gate
# Returns: Inplace modified QuantumCircuit

def SimpleAnsatz ( qc, q, n_qubits, theta_list, reps = 5, insert_barriers=False):
    for i in range (reps+1):
        theta = 0
        for j in range (n_qubits):
            #theta += theta_list[(i * n_qubits)+j]
            qc.ry(theta_list[(i * n_qubits)+j], q[j])

        if ( True == insert_barriers):
                qc.barrier()
    return

In [3]:
# Implements the ansatz circuit using RY rotations and CZ gates in a liner arrangement
# Inputs: qc – QuantumCircuit
# theta_list – rotational angle parameter must be (reps + 1) * n_qubits wide.
# q – the qubit register to use
# n_qubits – Number of qubits to use
# reps – repeats of the circuit, defaults to 5
# insert_barriers – whether to create a barrier gate after each rep.
# Returns: Inplace modified QuantumCircuit

def TwoLocalAnsatz ( qc, q, n_qubits, theta_list, reps=5, insert_barriers=False):
    i_theta = 0

    for i in range(n_qubits):
        qc.ry(theta_list[i_theta], q[i])
        i_theta += 1

    for _ in range(reps):
        if ( True == insert_barriers):
            qc.barrier()

        for i in range(n_qubits-1):
            qc.cz(q[i], q[i+1])

        for i in range(n_qubits):
            qc.ry(theta_list[i_theta], q[i])
            i_theta += 1
    return

In [4]:
# setup the Pauli operators corresponding to the problem Hamiltonian
# Inputs: nothing
# Returns: a list of Pauli operators and the corresponding weighing coefficients corresponding to the Hamiltonian


def get_Paulioperator():
    # Assume the following Hamiltonian
    #<H> = 2*<ψ(θ)|H1|ψ(θ)> + 4*<ψ(θ)|H2|ψ(θ)> + 8*<ψ(θ)|H3|ψ(θ)> + 16*<ψ(θ)|H4|ψ(θ)>
    #H1 = IIIZ
    #H2 = IIZZ
    #H3 = IZII
    #H4 = ZIIZ

    pauli_list = []
    w=[2+0j, 4+0j, 8+0j, 16+0j]

    pauli_list.append([w[0], "IIIZ"])
    pauli_list.append([w[1], "IIZZ"])
    pauli_list.append([w[2], "IZII"])
    pauli_list.append([w[3], "ZIIZ"])

    return pauli_list

In [5]:
# Performs pauli measurements
# qubitop: The Pauli operator
# param_dict: list of theta's for the parametrization circuit
# SHOTS: number of times, the circuit is to be repeated
def perform_pauli_measurements( qubitOp, param_dict, SHOTS=1024):

    avg = 0.0
    n_qubits = len(qubitOp[0][1])
    pauli_list = qubitOp


    # for each Pauli operator
    for p in pauli_list:
        weight = p[0].real
        pauli  = p[1]

        # assign parameters to the pqc and clone it
        qc = vqe_pqc.assign_parameters(param_dict)

        # Apply the Pauli operators.
        # no actions for "I" or "Z"
        for i in range(n_qubits):
            if (pauli[i] == "Y"):
                qc.sdg(q[i])
                qc.h(q[i])
            elif (pauli[i] == "X"):
                qc.h(q[i])

        # We should measure this circuit in the computation basis now
        qc.measure_all()

        job = backend.run(qc, shots= SHOTS, mode="sync", performance="HighestEfficiency", quiet=True)
        job_monitor(job, quiet=True)

        results = job.result()
        result_dict = results.get_counts()

        # perform the pauli measurement
        # convert the operator into binary
        measurement = 0.0
        pauli_int = int (p[1].replace("I","0").replace("Z","1").replace("X","1").replace("Y","1"),2)

        for key, value in result_dict.items():
            sign = -1.0 if ( bin(int(key,2) & pauli_int).count("1") & 1 ) else 1.0
            measurement += sign * value
        measurement /= SHOTS

        measurement = measurement * weight
        avg = avg + measurement

    return avg


# Given a theta_list, calculate the eigenstate
# Performs pauli measurements
# param_dict: list of theta's for the parametrization circuit
# SHOTS: number of times, the circuit is to be repeated
def find_eigenstate (param_dict, SHOTS=1024 ):

    # assign parameters to the pqc and clone it
    qc = vqe_pqc.assign_parameters(param_dict)

    # We should measure the circuit now
    qc.measure_all()

    job = backend.run(qc, shots= SHOTS, mode="sync", performance="HighestEfficiency", quiet=True)
    job_monitor(job, quiet=True)
    results = job.result()
    result_dict = results.get_counts()

    sorted_list = sorted(result_dict.items(), key=lambda value: value[1])

    return sorted_list[-1][0]

In [6]:
# Create the Parameterized Quantum Circuit
# Input:
# qubitop: The Pauli Operator
# theta_list: Parameter vector
# layers: number of circuit layers to use in the ansatz
# Returns: the Parameterized quantum circuit
# try both the ansatzes by uncommenting one of them

def create_ParameterizedVQECircuit(qubitOp, theta_list, layers):
    n_qubits = len(qubitOp[0][1])

    # construct the ansatz
    q = QuantumRegister(n_qubits, "q")
    c = ClassicalRegister(n_qubits, "c")
    circ = QuantumCircuit(q,c)

    #SimpleAnsatz (circ, q, n_qubits, theta_list, reps = layers)
    TwoLocalAnsatz (circ, q, n_qubits, theta_list, reps = layers)


    return circ

In [7]:
# Define the Pauli operator for the toy Hamiltonian described above
qubitOp = get_Paulioperator() # Setup the Pauli operators

# declare some constants
n_layers = 5 # Number of layers in the ansatz
n_calls = 300 # Number of optimizer iterations
n_qubits = len(qubitOp[0][1]) # Number of qubits required
n_params = (n_layers + 1) * n_qubits # Size of Parameter array
theta_list = ParameterVector("theta", n_params)

# Initial theta_list (parameters)
initial_parameters_list = []
for i in range (n_params):
    initial_parameters_list.append(uniform(-2*math.pi, 2*math.pi))

# create the Parameterized circuit
vqe_pqc = create_ParameterizedVQECircuit(qubitOp, theta_list, n_layers)

avg_list = []

# The VQE routine called by the optimizer
# theta_list: the parameter list to be tried in the current cycle

def vqe(param_list) -> float:
    # create the parameter dictionary
    param_dict = {}
    for index in range (0, n_params):
        param_dict[theta_list[index].name()] = param_list[index]

    avg = perform_pauli_measurements (qubitOp, param_dict)
    #print(theta_list)
    #print(avg)
    avg_list.append(avg)
    return avg

# Method to find the eigenstate, for a given theta_list
# theta_list: The parameter array
def vqe_eigenstate(param_list):
    # create the parameter dictionary
    param_dict = {}
    for index in range (0, n_params):
        param_dict[theta_list[index].name()] = param_list[index]

    return find_eigenstate(param_dict)

print("Using scipy minimize, method: COBYLA:")

vqe_result_scipy = scipy.optimize.minimize(vqe, initial_parameters_list, method="COBYLA")

print ("The estimated ground state energy: ", vqe_result_scipy.fun)

eigenstate_scipy = vqe_eigenstate(vqe_result_scipy.x)
print ("Eigenstate: ", eigenstate_scipy)

Using scipy minimize, method: COBYLA:
The estimated ground state energy:  -27.50390625
Eigenstate:  0101
